<a href="https://colab.research.google.com/github/16suryanshrastogi/NextWordPrediction/blob/main/Nextwordpridiction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Read the text file
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [ ]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [ ]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
3010/3010 [==============================] - 217s 71ms/step - loss: 6.2277 - accuracy: 0.0782
Epoch 2/50
3010/3010 [==============================] - 210s 70ms/step - loss: 5.4917 - accuracy: 0.1251
Epoch 3/50
3010/3010 [==============================] - 209s 69ms/step - loss: 5.1084 - accuracy: 0.1468
Epoch 4/50
3010/3010 [==============================] - 204s 68ms/step - loss: 4.7770 - accuracy: 0.1647
Epoch 5/50
3010/3010 [==============================] - 205s 68ms/step - loss: 4.4726 - accuracy: 0.1845
Epoch 6/50
3010/3010 [==============================] - 209s 69ms/step - loss: 4.1889 - accuracy: 0.2046
Epoch 7/50
3010/3010 [==============================] - 205s 68ms/step - loss: 3.9183 - accuracy: 0.2302
Epoch 8/50
3010/3010 [==============================] - 209s 69ms/step - loss: 3.6613 - accuracy: 0.2605
Epoch 9/50
3010/3010 [==============================] - 211s 70ms/step - loss: 3.4176 - accuracy: 0.2948
Epoch 10/50
3010/3010 [==============================] 